# 4. Quatization

#### 문제

Quantization (양자화) 과정은, 더 넓은 범위를 갖는 값들을 작은 범위를 갖는 값들로 근사해 표현함으로써 자료를 손실 압축하는 과정을 말한다. 예를 들어 16비트 JPG 파일을 4컬러 GIF 파일로 변환하는 것은 RGB 색 공간의 색들을 4컬러 중의 하나로 양자화하는 것이고, 키가 161, 164, 170, 178 인 학생 넷을 '160대 둘, 170대 둘' 이라고 축약해 표현하는 것 또한 양자화라고 할 수 있다.

1000 이하의 자연수들로 구성된 수열을 최대 S종류 의 값만을 사용하도록 양자화하고 싶다. **이 때 양자화된 숫자는 원래 수열에 없는 숫자일 수도 있다.** 양자화를 하는 방법은 여러 가지가 있다. 수열 1 2 3 4 5 6 7 8 9 10 을 2개의 숫자만을 써서 표현하려면, 3 3 3 3 3 7 7 7 7 7 과 같이 할 수도 있고, 1 1 1 1 1 10 10 10 10 10 으로 할 수도 있다. 우리는 이 중, 각 숫자별 오차 제곱의 합을 최소화하는 양자화 결과를 알고 싶다.

예를 들어, 수열 1 2 3 4 5 를 1 1 3 3 3 으로 양자화하면 오차 제곱의 합은 0+1+0+1+4=6 이 되고, 2 2 2 4 4 로 양자화하면 오차 제곱의 합은 1+0+1+0+1=3 이 된다.

수열과 S 가 주어질 때, 가능한 오차 제곱의 합의 최소값을 구하는 프로그램을 작성하시오.



#### 입력

입력의 첫 줄에는 테스트 케이스의 수 C (1 <= C <= 50) 가 주어진다. 각 테스트 케이스의 첫 줄에는 수열의 길이 N (1 <= N <= 100), 사용할 숫자의 수 S (1 <= S <= 10) 이 주어진다. 그 다음 줄에 N개의 정수로 수열의 숫자들이 주어진다. 수열의 모든 수는 1000 이하의 자연수이다.



#### 출력

각 테스트 케이스마다, 주어진 수열을 최대 S 개의 수로 양자화할 때 오차 제곱의 합의 최소값을 출력한다.



#### 예제 입력

```
2
10 3
3 3 3 1 2  3 2 2 2 1
9 3
1 744 755 4 897 902 890 6 777
```



#### 예제 출력

```
0
651
```

In [11]:
def get_squared_error(seq, quantized_seq):
    '''quantized_seq가 겹치지 않는 숫자로 이루어져있을 때를 가정'''
    S = len(quantized_seq)
    i = 0
    
    errors = []
    for s in seq:
        if i != S - 1 and abs(s - quantized_seq[i]) > abs(s - quantized_seq[i+1]):
            i += 1
        errors.append((s - quantized_seq[i]) ** 2)
        
    return sum(errors)


def get_squared_error_v2(seq, quantized_seq):
    '''seq와 quantized_seq가 동일한 길이일 때를 가정'''
    return sum([(s - qs) ** 2 for s, qs in zip(seq, quantized_seq)])

자 보자. 결국 S개의 quantized로 나눈다는 것은, n개의 숫자를 1개의 숫자로 바꾼다는 것

unique한 숫자들의 배열이 있는 상태에서, 중간에 결합할 지 말지를 permutate하는 방식으로 하면 된다.

In [35]:
from itertools import permutations

In [36]:
for s in set(permutations([0,0,0,1,1])):
    print(s)

(0, 0, 1, 1, 0)
(1, 0, 1, 0, 0)
(1, 1, 0, 0, 0)
(0, 0, 1, 0, 1)
(0, 0, 0, 1, 1)
(0, 1, 1, 0, 0)
(0, 1, 0, 1, 0)
(1, 0, 0, 1, 0)
(0, 1, 0, 0, 1)
(1, 0, 0, 0, 1)


In [61]:
def quantize(seq, connect_permutation):
    answer = []
    connected = [seq[0]]
    for i in range(len(connect_permutation)):
        if connect_permutation[i] == 0:
            answer.append(round(np.mean(connected), 1))
            connected = []
        connected.append(seq[i+1])        
    answer.append(round(np.mean(connected), 1))
    return list(map(int, answer))

In [62]:
quantize([1,2,3,4,5], [0,0,1,1])

[1, 2, 4]

In [63]:
def solution(seq, S):
    seq = sorted(seq)
    unique_numbers = sorted(list(set(seq)))
    
    # sequence의 고유 숫자 수가 S 이하면 quantize안해도 된다
    if len(unique_numbers) <= S:
        return 0
    
    # 한 개면 그냥 바로 계산
    if S == 1:
        mean = int(np.mean(seq))
        return sum([(s - mean) ** 2 for s in seq])
    
    # S가 2 이상이면, 무조건 S개만큼 사용하는 것이 error가 작다
    # connect할 갯수는 고유 숫자 수 - S
    connect_permutations = set(permutations([0] * (S - 1) + [1] * (len(unique_numbers) - S)))
    
    answer = np.inf
    for connect_permutation in connect_permutations:
        quantized_seq = quantize(unique_numbers, connect_permutation)
        print(seq, quantized_seq)
        answer = min(answer, get_squared_error(seq, quantized_seq))
    return answer

In [64]:
solution([3, 3, 3, 1, 2, 3, 2, 2, 2, 1], 2)

[1, 1, 2, 2, 2, 2, 3, 3, 3, 3] [1, 2]
[1, 1, 2, 2, 2, 2, 3, 3, 3, 3] [1, 3]


4